In [1]:
import pandas as pd
import os
import tqdm
import math
import numpy as np
import datatable as dt

In [2]:
df = pd.read_feather('datasets/KT3_merged.feather')

In [11]:
questions_df = pd.read_csv('datasets/contents/questions.csv')

In [12]:
questions_df.head()

,question_id,bundle_id,explanation_id,correct_answer,part,tags,deployed_at
0,q1,b1,e1,b,1,1;2;179;181,1558093217098
1,q2,b2,e2,a,1,15;2;182,1558093219720
2,q3,b3,e3,b,1,14;2;179;183,1558093222784
3,q4,b4,e4,b,1,9;2;179;184,1558093225357
4,q5,b5,e5,c,1,8;2;179;181,1558093228439


In [3]:
df.head()

,timestamp,action_type,item_id,source,user_answer,platform,user_id
0,1565096151269,enter,b3544,diagnosis,,mobile,1
1,1565096187972,respond,q5012,diagnosis,b,mobile,1
2,1565096194904,submit,b3544,diagnosis,,mobile,1
3,1565096195001,enter,b3238,diagnosis,,mobile,1
4,1565096218682,respond,q4706,diagnosis,c,mobile,1


In [4]:
df['user_id'].unique()

array([     1,      3,      4, ..., 834027, 834443, 837094], dtype=int32)

### Keep first response for each question

In [23]:
answers_dict = {q: questions_df[questions_df['question_id']==q]['correct_answer'].to_numpy()[0]
               for q in questions_df['question_id']}

In [41]:
df_resp = df[df['action_type']=='respond']

In [13]:
df_first_resp = df.drop_duplicates(['user_id', 'item_id'], keep='first')
df_first_resp = df_first_resp[df_first_resp['action_type']=='respond']

In [30]:
df_first_resp['correct'] = 0
df_first_resp.head()

,timestamp,action_type,item_id,source,user_answer,platform,user_id,correct
1,1565096187972,respond,q5012,diagnosis,b,mobile,1,0
4,1565096218682,respond,q4706,diagnosis,c,mobile,1,0
7,1565096290094,respond,q4366,diagnosis,b,mobile,1,0
10,1565096337361,respond,q4829,diagnosis,a,mobile,1,0
13,1565096395328,respond,q6528,diagnosis,b,mobile,1,0


In [42]:
df_resp

,timestamp,action_type,item_id,source,user_answer,platform,user_id
1,1565096187972,respond,q5012,diagnosis,b,mobile,1
4,1565096218682,respond,q4706,diagnosis,c,mobile,1
7,1565096290094,respond,q4366,diagnosis,b,mobile,1
10,1565096337361,respond,q4829,diagnosis,a,mobile,1
13,1565096395328,respond,q6528,diagnosis,b,mobile,1
...,...,...,...,...,...,...,...
89270638,1574239015301,respond,q784,sprint,c,mobile,837094
89270639,1574239020870,respond,q784,sprint,b,mobile,837094
89270640,1574239021240,respond,q784,sprint,a,mobile,837094
89270645,1574240861444,respond,q7284,sprint,b,mobile,837094


In [43]:
df_first_resp.head(20)

,timestamp,action_type,item_id,source,user_answer,platform,user_id,correct
1,1565096187972,respond,q5012,diagnosis,b,mobile,1,False
4,1565096218682,respond,q4706,diagnosis,c,mobile,1,True
7,1565096290094,respond,q4366,diagnosis,b,mobile,1,True
10,1565096337361,respond,q4829,diagnosis,a,mobile,1,False
13,1565096395328,respond,q6528,diagnosis,b,mobile,1,False
16,1565096461374,respond,q4793,diagnosis,a,mobile,1,False
19,1565096499772,respond,q6488,diagnosis,a,mobile,1,True
24,1565097095356,respond,q356,sprint,b,mobile,1,True
29,1565097168971,respond,q1382,sprint,c,mobile,1,True
34,1565097238650,respond,q830,sprint,b,mobile,1,True


In [45]:
df_first_resp['user_answer']==questions_df[questions_df['question_id']==df_first_resp['item_id']]['correct_answer']

ValueError: Can only compare identically-labeled Series objects

In [ ]:
questions_df.loc[df_first_resp['item_id']]